# 7.2 torchtext에서의 Dataset, DataLoader 구현 방법

- torchtext를 사용하여 Dataset 및 DataLoader을 구현하는 방법을 설명합니다.


※　이 장의 파일은 Ubuntu 환경에서의 동작을 전제로 하고 있습니다. Windows와 같이 문자 코드가 다른 환경에서는 동작에 주의하십시오.

# 7.2 학습 목표

1.	torchtext를 이용하여 Dataset 및 DataLoader를 구현할 수 있다

# 사전 준비

- 도서의 지시에 따라, 이 장에서 사용하는 데이터를 준비합니다

- torchtext를 설치합니다

- pip install torchtext



# 1 . 전처리 및 단어 분할 함수 구현


In [1]:
# 단어 분할에는 Janome을 사용
from janome.tokenizer import Tokenizer

j_t = Tokenizer()


def tokenizer_janome(text):
    return [tok for tok in j_t.tokenize(text, wakati=True)]


In [2]:
# 전처리로서, 정규화 함수를 정의
import re

def preprocessing_text(text):
    # 반각 및 전각의 통일
    # 이번에는 무시

    # 영어의 소문자화
    # 이번에는 무시
    # output = output.lower()

    # 줄 바꿈, 반각 스페이스, 전각 스페이스를 삭제
    text = re.sub('\r', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('　', '', text)
    text = re.sub(' ', '', text)

    # 숫자를 일률적으로 "0"로 설정
    text = re.sub(r'[0-9 ０-９]', '0', text)  # 숫자

    # 기호와 숫자 제거
    # 이번에는 무시. 반각 기호, 숫자, 영자(英字)
    # 이번에는 무시. 전각 기호

    # 특정 문자를 정규 표현으로 치환
    # 이번에는 무시

    return text


In [3]:
# 전처리 및 Janome의 단어 분할을 수행하는 함수를 정의한다
def tokenizer_with_preprocessing(text):
    text = preprocessing_text(text)  # 전처리 정규화
    ret = tokenizer_janome(text)  # Janome 단어 분할

    return ret


# 동작 확인
text = "昨日は とても暑く, 気温が36度もあった。"
print(tokenizer_with_preprocessing(text))


['昨日', 'は', 'とても', '暑く', ', ', '気温', 'が', '00', '度', 'も', 'あっ', 'た', '。']


# 2. 문장 데이터 읽기

In [4]:
import torchtext

# tsv나 csv 데이터를 읽을 때, 읽어들인 내용에 대해 수행할 처리를 정의합니다
# 문장과 라벨을 모두 준비합니다

max_length = 25
TEXT = torchtext.data.Field(sequential=True, tokenize=tokenizer_with_preprocessing,
                            use_vocab=True, lower=True, include_lengths=True, batch_first=True, fix_length=max_length)
LABEL = torchtext.data.Field(sequential=False, use_vocab=False)

# 인수의 의미는 다음과 같습니다
# sequential: 데이터의 길이가 가변인가? 문장은 길이가 다양하므로 True. 라벨은 False
# tokenize: 문장을 읽을 때, 전처리 및 단어 분할 함수를 정의
# use_vocab: 단어를 vocabulary(단어집: 이후에 설명)에 추가할지 여부
# lower: 알파벳이 존재할 때 소문자로 변환할지 여부
# include_length: 문장의 단어 수 데이터를 포함할지 여부
# batch_first: 미니 배치 차원을 선두에 제공할지 여부
# fix_length: 전체 문장을 지정한 길이가 되도록 padding


In [5]:
# data.TabularDataset의 상세 정보
# https://torchtext.readthedocs.io/en/latest/examples.html?highlight=data.TabularDataset.splits

# "data" 폴더에서 각 tsv 파일을 읽어들여, Dataset으로 합니다
# 1행이 TEXT와 LABEL로 구분되었음을 fields에서 지시합니다
train_ds, val_ds, test_ds = torchtext.data.TabularDataset.splits(
    path='./data/', train='text_train.tsv',
    validation='text_val.tsv', test='text_test.tsv', format='tsv',
    fields=[('Text', TEXT), ('Label', LABEL)])


# 동작 확인
print('훈련 데이터 수', len(train_ds))
print('첫번째 훈련 데이터', vars(train_ds[0]))
print('두번째 훈련 데이터', vars(train_ds[1]))


훈련 데이터 수 4
첫번째 훈련 데이터 {'Text': ['王', 'と', '王子', 'と', '女王', 'と', '姫', 'と', '男性', 'と', '女性', 'が', 'い', 'まし', 'た', '。'], 'Label': '0'}
두번째 훈련 데이터 {'Text': ['機械', '学習', 'が', '好き', 'です', '。'], 'Label': '1'}


# 단어의 수치화

In [6]:
# vocabulary를 작성합니다
# 훈련 데이터 train의 단어에서 min_freq 이상의 빈도 단어를 사용하여 vocabulary(단어집)를 구축
TEXT.build_vocab(train_ds, min_freq=1)

# 훈련 데이터의 단어와 빈도를 출력(빈도 min_freq보다 큰 것이 출력됩니다)
TEXT.vocab.freqs  # 출력하기


Counter({'王': 1,
         'と': 5,
         '王子': 1,
         '女王': 1,
         '姫': 1,
         '男性': 1,
         '女性': 1,
         'が': 3,
         'い': 1,
         'まし': 1,
         'た': 1,
         '。': 4,
         '機械': 1,
         '学習': 1,
         '好き': 1,
         'です': 1,
         '本章': 2,
         'から': 1,
         '自然': 1,
         '言語': 1,
         '処理': 1,
         'に': 1,
         '取り組み': 1,
         'ます': 2,
         'で': 1,
         'は': 1,
         '商品': 1,
         'レビュー': 1,
         'の': 4,
         '短い': 1,
         '文章': 4,
         'に対して': 1,
         ', ': 3,
         'その': 1,
         'negative': 1,
         'な': 4,
         '評価': 2,
         'を': 3,
         'し': 3,
         'て': 2,
         'いる': 2,
         'か': 2,
         'positive': 1,
         '0': 1,
         '値': 1,
         'クラス': 1,
         '分類': 2,
         'する': 1,
         'モデル': 1,
         '構築': 1})

In [7]:
# vocabulary의 단어를 id로 변환한 결과를 출력.
# 빈도가 min_freq보다 작은 경우에는 알 수 없음<unk>이 된다

TEXT.vocab.stoi  # 출력. string to identifiers 문자열을 id로


defaultdict(<function torchtext.vocab._default_unk_index()>,
            {'<unk>': 0,
             '<pad>': 1,
             'と': 2,
             '。': 3,
             'な': 4,
             'の': 5,
             '文章': 6,
             ', ': 7,
             'が': 8,
             'し': 9,
             'を': 10,
             'いる': 11,
             'か': 12,
             'て': 13,
             'ます': 14,
             '分類': 15,
             '本章': 16,
             '評価': 17,
             '0': 18,
             'い': 19,
             'から': 20,
             'する': 21,
             'その': 22,
             'た': 23,
             'で': 24,
             'です': 25,
             'に': 26,
             'に対して': 27,
             'は': 28,
             'まし': 29,
             'クラス': 30,
             'negative': 31,
             'positive': 32,
             'モデル': 33,
             'レビュー': 34,
             '値': 35,
             '処理': 36,
             '取り組み': 37,
             '商品': 38,
             '女性': 39,
             '女王': 

# DataLoader 작성

In [8]:
# DataLoader를 작성합니다(torchtext의 맥락에서는 단순히 iterater로 부릅니다)
train_dl = torchtext.data.Iterator(train_ds, batch_size=2, train=True)

val_dl = torchtext.data.Iterator(
    val_ds, batch_size=2, train=False, sort=False)

test_dl = torchtext.data.Iterator(
    test_ds, batch_size=2, train=False, sort=False)


# 동작 확인 - 검증 데이터의 데이터 세트로 확인
batch = next(iter(val_dl))
print(batch.Text)
print(batch.Label)


(tensor([[46,  2, 47,  2, 40,  2, 42,  2, 48,  2, 39,  8, 19, 29, 23,  3,  1,  1,
          1,  1,  1,  1,  1,  1,  1],
        [45, 43,  8, 41, 25,  3,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
          1,  1,  1,  1,  1,  1,  1]]), tensor([16,  6]))
tensor([0, 1])


끝